### Load Model

In [1]:
import lightning as L
import torch
from tools import PyTorchMLP, LightningModel, MNISTDataModule

In [2]:
pt_model = PyTorchMLP(num_features=784, num_classes=10)

lt_model = LightningModel.load_from_checkpoint(
    checkpoint_path="lt_model.ckpt", model=pt_model
)

In [3]:
d_module = MNISTDataModule()
d_module.setup(stage="test")

### Use new data for prediction

In [4]:
import torchmetrics

test_dataloader = d_module.test_dataloader()
acc = torchmetrics.Accuracy(task="multiclass", num_classes=10)

In [5]:
for batch in test_dataloader:

    features, true_labels = batch

    with torch.inference_mode():
        logits = lt_model(features)

    predicted_labels = torch.argmax(logits, dim=1)
    acc(predicted_labels, true_labels)

predicted_labels[:5]

tensor([1, 2, 3, 4, 5])

In [6]:
test_acc = acc.compute()
print(f"test accuracy: {test_acc * 100:.2f}%")

test accuracy: 96.53%
